In [24]:
import sqlite3
import pandas as pd
import time
from tensorflow.keras.models import load_model
import numpy as np

# Connect to the database
db_path = "../prisma/dev.db"
conn = sqlite3.connect(db_path)

library_name = "Bibliotheks­bereich A3"
last_chunk = None

# Load pre-trained RNN model
# model = load_model("path_to_your_model.h5")

In [29]:
def fetch_latest_data():
    global last_chunk
    query = f"""
        SELECT year, month, day, chunk, percentage
        FROM BibData
        WHERE name = '{library_name}' 
    """
    df = pd.read_sql(query, conn)
    if not df.empty:
        last_chunk = df['chunk'].iloc[-1]  # Update to latest chunk
    return df

def preprocess_data(df):
    return np.array(df['percentage']).reshape(-1, 1)  # Shape for RNN input

In [30]:
fetch_latest_data()

,year,month,day,chunk,percentage
0,2024,6,2,109,100
1,2024,6,2,109,100
2,2024,6,2,110,100
3,2024,6,2,110,100
4,2024,6,2,111,100
...,...,...,...,...,...
27532,2024,12,11,96,100
27533,2024,12,11,97,100
27534,2024,12,11,98,100
27535,2024,12,11,99,98


In [33]:
# Sort the DataFrame chronologically
df = df.sort_values(by=['year', 'month', 'day', 'chunk']).reset_index(drop=True)
df

,year,month,day,chunk,percentage
0,2024,6,2,0,28
1,2024,6,2,1,25
2,2024,6,2,2,25
3,2024,6,2,3,24
4,2024,6,2,4,23
...,...,...,...,...,...
27532,2024,12,11,96,100
27533,2024,12,11,97,100
27534,2024,12,11,98,100
27535,2024,12,11,99,98


In [ ]:
# create the chunks, we are gonna take 6 chunks of data = 1 hour
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])  # Take window_size chunks as features
        y.append(data[i+window_size])    # Predict the next 'percentage' value
    return np.array(X), np.array(y)

window_size = 6
X, y = create_sequences(df['percentage'].values, window_size)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

model = Sequential([
    GRU(64, input_shape=(window_size, 1), return_sequences=False),
    Dense(1)  # Predict next chunk percentage
])

model.compile(optimizer="adam", loss="mse")
model.fit(X.reshape(-1, window_size, 1), y, epochs=10, batch_size=16)


In [ ]:
### Change this?
latest_sequence = df['percentage'].values[-window_size:].reshape(1, window_size, 1)
next_prediction = model.predict(latest_sequence)
print("Next chunk prediction:", next_prediction)


In [ ]:
while True:
    data = fetch_latest_data()
    if not data.empty:
        preprocessed_data = preprocess_data(data)
        predictions = model.predict(preprocessed_data)
        print(f"Latest Prediction for {library_name}:", predictions)
    time.sleep(10)  # Polling every 10 minutes